

---



---
## Developed Dataset to show working


---



---




In [ ]:
pd.read_csv("/content/form_data.csv")

,CUST_ID,age,workclass,education,educational-num,marital-status,occupation,gender,hours-per-week,native-country,income,city
0,C10001,25,Private,11th,7,Never-married,Machine-op-inspct,Male,40,India,307000,Vizag
1,C10002,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Male,50,India,115000,Bengaluru
2,C10003,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Male,40,India,188000,Vizag
3,C10004,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Male,40,India,468000,Ahmedabad
4,C10005,18,Private,Some-college,10,Never-married,Prof-specialty,Female,30,India,472000,Ahmedabad
...,...,...,...,...,...,...,...,...,...,...,...,...
8945,C19186,39,Self-emp-inc,Some-college,10,Married-civ-spouse,Exec-managerial,Male,84,India,416000,Delhi
8946,C19187,52,Private,HS-grad,9,Married-civ-spouse,Tech-support,Male,40,India,354000,Mumbai
8947,C19188,21,State-gov,HS-grad,9,Never-married,Other-service,Female,40,India,310000,Lucknow
8948,C19189,57,Self-emp-not-inc,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Male,70,India,76000,Lucknow



### Preparing some Test examples





In [180]:
N = np.random.randint(1000,8000)
test = data.iloc[N:N+5,1:].reset_index().drop(['index'],axis=1)
test = pd.concat([pd.DataFrame({'CUST_ID':['D21000','D21001','D21002','D21003','D21004']}),test], axis=1)
test

,CUST_ID,age,workclass,education,educational-num,marital-status,occupation,gender,hours-per-week,native-country,income,city
0,D21000,46,0,HS-grad,9,Married-civ-spouse,Transport-moving,Male,60,India,340000,Bengaluru
1,D21001,36,0,1st-4th,2,Married-civ-spouse,Other-service,Male,40,Mexico,206000,Mumbai
2,D21002,45,4,Bachelors,13,Married-civ-spouse,Adm-clerical,Male,38,India,126000,Lucknow
3,D21003,19,4,Some-college,10,Never-married,Other-service,Male,20,India,278000,Patna
4,D21004,38,0,Assoc-acdm,12,Never-married,Other-service,Female,25,India,120000,Delhi




---



---
# CODE


---



---




In [181]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings(action="ignore")

import os

In [204]:
class member_updater():

  def access_proc_data(self, path):
    self.procdf = pd.read_csv(path)
    return self.procdf.head()
  
  def access_form_data(self, path):
    self.formdf = pd.read_csv(path)
    return self.formdf.head()

  def train(self):
    self.data = self.formdf.copy()
    self.data.workclass = pd.factorize(self.data.workclass)[0]
    self.data.education = pd.factorize(self.data.education)[0]
    self.data['marital-status'] = pd.factorize(self.data['marital-status'])[0]
    self.data.occupation = pd.factorize(self.data.occupation)[0]
    self.data.gender = pd.factorize(self.data.gender)[0]
    self.data['native-country'] = pd.factorize(self.data['native-country'])[0]
    self.data.city = pd.factorize(self.data.city)[0]
    
    X = np.array(self.data)[:,1:]
    self.nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X)

  def predict(self, test):
    test.workclass = pd.factorize(test.workclass)[0]
    test.education = pd.factorize(test.education)[0]
    test['marital-status'] = pd.factorize(test['marital-status'])[0]
    test.occupation = pd.factorize(test.occupation)[0]
    test.gender = pd.factorize(test.gender)[0]
    test['native-country'] = pd.factorize(test['native-country'])[0]
    test.city = pd.factorize(test.city)[0]
    distances, indices = self.nbrs.kneighbors(test.iloc[:,1:])
    self.dff = pd.DataFrame()
    for x in indices:
      self.dff = pd.concat([self.dff, self.procdf.iloc[x,1:]], axis=0)
    self.dff = pd.concat([pd.DataFrame(test.iloc[:,0]),self.dff.reset_index().drop(['index'],axis=1)], axis=1)
    return self.dff

  def update(self, save_filepath):
    newdf = pd.concat([self.procdf, self.dff], axis=0)
    newdf.to_csv(save_filepath, index=None)
 




---



---


# How to Use:


---



---



In [205]:
obj = member_updater()

In [206]:
obj.access_proc_data('/content/processed_data.csv')

,CUST_ID,TENURE,BALANCE_RANGE,PURCHASES_RANGE,ONEOFF_PURCHASES_RANGE,INSTALLMENTS_PURCHASES_RANGE,CASH_ADVANCE_RANGE,CREDIT_LIMIT_RANGE,PAYMENTS_RANGE,MINIMUM_PAYMENTS_RANGE,BALANCE_FREQUENCY_RANGE,PURCHASES_FREQUENCY_RANGE,ONEOFF_PURCHASES_FREQUENCY_RANGE,PURCHASES_INSTALLMENTS_FREQUENCY_RANGE,CASH_ADVANCE_FREQUENCY_RANGE,PRC_FULL_PAYMENT_RANGE,PURCHASES_TRX_RANGE,CASH_ADVANCE_TRX_RANGE
0,C10001,12,1,1,0,1,0,2,1,1,9,2,0,1,0,0,1,0
1,C10002,12,4,0,0,0,5,5,4,3,10,0,0,0,3,3,0,1
2,C10003,12,3,2,2,0,0,5,2,2,10,10,10,0,0,0,3,0
3,C10004,12,3,3,3,0,1,5,0,2,7,1,1,0,1,0,1,1
4,C10005,12,2,1,1,0,0,3,2,1,10,1,1,0,0,0,1,0


In [207]:
obj.access_form_data('/content/form_data.csv')

,CUST_ID,age,workclass,education,educational-num,marital-status,occupation,gender,hours-per-week,native-country,income,city
0,C10001,25,Private,11th,7,Never-married,Machine-op-inspct,Male,40,India,307000,Vizag
1,C10002,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Male,50,India,115000,Bengaluru
2,C10003,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Male,40,India,188000,Vizag
3,C10004,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Male,40,India,468000,Ahmedabad
4,C10005,18,Private,Some-college,10,Never-married,Prof-specialty,Female,30,India,472000,Ahmedabad


In [208]:
obj.train()

In [209]:
test

,CUST_ID,age,workclass,education,educational-num,marital-status,occupation,gender,hours-per-week,native-country,income,city
0,D21000,46,0,0,9,0,0,0,60,0,340000,0
1,D21001,36,0,1,2,0,1,0,40,1,206000,1
2,D21002,45,1,2,13,0,2,0,38,0,126000,2
3,D21003,19,1,3,10,1,1,0,20,0,278000,3
4,D21004,38,0,4,12,1,1,1,25,0,120000,4


In [210]:
obj.predict(test)

,CUST_ID,TENURE,BALANCE_RANGE,PURCHASES_RANGE,ONEOFF_PURCHASES_RANGE,INSTALLMENTS_PURCHASES_RANGE,CASH_ADVANCE_RANGE,CREDIT_LIMIT_RANGE,PAYMENTS_RANGE,MINIMUM_PAYMENTS_RANGE,BALANCE_FREQUENCY_RANGE,PURCHASES_FREQUENCY_RANGE,ONEOFF_PURCHASES_FREQUENCY_RANGE,PURCHASES_INSTALLMENTS_FREQUENCY_RANGE,CASH_ADVANCE_FREQUENCY_RANGE,PRC_FULL_PAYMENT_RANGE,PURCHASES_TRX_RANGE,CASH_ADVANCE_TRX_RANGE
0,D21000,11,3,1,0,1,0,5,1,1,10,1,0,1,0,0,1,0
1,D21001,12,5,3,2,3,5,5,3,3,10,10,6,9,7,0,6,5
2,D21002,12,1,1,1,0,0,3,2,2,2,1,1,0,0,0,1,0
3,D21003,12,1,1,0,1,0,4,2,1,10,9,0,9,0,4,3,0
4,D21004,12,1,2,1,1,0,3,2,1,10,9,1,8,0,0,3,0


In [211]:
obj.update('/content/updated_data.csv')